In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader


In [16]:
# Define transformations for training and testing data
train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])
# Load MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=train_transforms)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=test_transforms)

# Create data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)


Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 9.91M/9.91M [00:01<00:00, 5.70MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 28.9k/28.9k [00:00<00:00, 145kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 1.65M/1.65M [00:01<00:00, 1.38MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 4.54k/4.54k [00:00<00:00, 3.47MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [17]:
# Define the CNN model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)  # 1 input channel for grayscale
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(128 * 3 * 3, 256)  # Adjusted for MNIST image size

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 3 * 3)
        x = F.relu(self.fc1(x))
        return x


In [18]:
# Define the ArcFace layer
class ArcFaceLayer(nn.Module):
    def __init__(self, num_classes, s=30.0, m=0.50):
        super(ArcFaceLayer, self).__init__()
        self.num_classes = num_classes
        self.s = s
        self.m = m
        self.W = nn.Parameter(torch.empty(num_classes, 256))
        nn.init.xavier_uniform_(self.W.data)

    def forward(self, x, labels):
        x = F.normalize(x)
        W = F.normalize(self.W, dim=1)
        cos_theta = F.linear(x, W)
        return cos_theta


In [19]:
# Combine CNN and ArcFace layers
class ArcFaceModel(nn.Module):
    def __init__(self, num_classes):
        super(ArcFaceModel, self).__init__()
        self.cnn = CNNModel()
        self.arcface = ArcFaceLayer(num_classes)

    def forward(self, x, labels):
        x = self.cnn(x)
        x = self.arcface(x, labels)
        return x


In [20]:
num_epochs = 10


In [21]:
# Training loop with accuracy calculation
model = ArcFaceModel(num_classes=10)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    model.train()
    correct = 0
    total = 0

    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images, labels)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = 100 * correct / total
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, Training Accuracy: {train_accuracy:.2f}%')


Epoch [1/10], Loss: 1.4236, Training Accuracy: 96.90%
Epoch [2/10], Loss: 1.4198, Training Accuracy: 98.97%
Epoch [3/10], Loss: 1.4188, Training Accuracy: 99.17%
Epoch [4/10], Loss: 1.4310, Training Accuracy: 99.39%
Epoch [5/10], Loss: 1.4197, Training Accuracy: 99.56%
Epoch [6/10], Loss: 1.4241, Training Accuracy: 99.55%
Epoch [7/10], Loss: 1.4148, Training Accuracy: 99.65%
Epoch [8/10], Loss: 1.4213, Training Accuracy: 99.69%
Epoch [9/10], Loss: 1.4231, Training Accuracy: 99.72%
Epoch [10/10], Loss: 1.4193, Training Accuracy: 99.71%


In [22]:
# Evaluate the model
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images, labels)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')


Accuracy on test set: 99.36%
